## 1. Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Scaling
from sklearn.preprocessing import MinMaxScaler

 #Correlation Heatmap
from matplotlib.colors import LinearSegmentedColormap

#Statistical Test
from scipy import stats
from sklearn.impute import SimpleImputer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None)

In [2]:
# # You can download the data in the source that is linked above the table of contents

# # Read in the data
# X_train_encoded = pd.read_csv('../project_data/X_train_encoded.csv', delimiter=',', index_col=0)
# X_val_encoded = pd.read_csv('../project_data/X_val_encoded.csv', delimiter=',', index_col=0)

# y_train = pd.read_csv('../project_data/y_train.csv',delimiter=',', index_col=0)
# y_val= pd.read_csv('../project_data/y_val.csv', delimiter=',', index_col=0)

# X_test_encoded = pd.read_csv('../project_data/X_test_encoded.csv',index_col=0)

In [3]:
# You can download the data in the source that is linked above the table of contents

# Read in the data
X_train_encoded_bin = pd.read_csv('../project_data/X_train_encoded_binary.csv', delimiter=',', index_col=0)
X_val_encoded_bin = pd.read_csv('../project_data/X_val_encoded_binary.csv', delimiter=',', index_col=0)

y_train_bin = pd.read_csv('../project_data/y_train_binary.csv',delimiter=',', index_col=0)
y_val_bin= pd.read_csv('../project_data/y_val_binary.csv', delimiter=',', index_col=0)

X_test_encoded_bin = pd.read_csv('../project_data/X_test_encoded_binary.csv',index_col=0)

## 2. Preparing the data

### 2.1 Encoding target variable

In [4]:
# claim_injury_type_mapping = {
#     '4. TEMPORARY': 3,
#     '2. NON-COMP': 1,
#     '5. PPD SCH LOSS': 4,
#     '3. MED ONLY': 2,
#     '6. PPD NSL': 5,
#     '1. CANCELLED': 0,
#     '8. DEATH':7,
#     '7. PTD': 6
# }

# y_train_encoded = y_train['Claim Injury Type'].map(claim_injury_type_mapping)
# y_val_encoded = y_val['Claim Injury Type'].map(claim_injury_type_mapping)

In [4]:
y_train_bin = y_train_bin.squeeze()  # Converts a DataFrame with one column to a Series

### 2.2 Separate Numerical and Categorical

In [5]:
X_train_encoded_bin.columns

Index(['Age at Injury', 'IME-4 Count', 'Number of Dependents',
       'Alternative Dispute Resolution_binary', 'COVID-19 Indicator_binary',
       'Attorney/Representative_binary', 'Industry Code_encoded_0',
       'Industry Code_encoded_1', 'WCIO Cause of Injury Code_encoded_0',
       'WCIO Cause of Injury Code_encoded_1',
       'WCIO Nature of Injury Code_encoded_0',
       'WCIO Nature of Injury Code_encoded_1',
       'WCIO Part Of Body Code_encoded_0', 'WCIO Part Of Body Code_encoded_1',
       'Industry Code_freq', 'WCIO Cause of Injury Code_freq',
       'WCIO Nature of Injury Code_freq', 'WCIO Part Of Body Code_freq',
       'Carrier Type Imputed_freq', 'Carrier Name_freq',
       'Carrier Type_Self-insured Private Entity',
       'Carrier Type_Self-insured Public Entity', 'Carrier Type_Special Funds',
       'Carrier Type_State Insurance Fund', 'Carrier Type_nan',
       'C-3 Date_nabinary', 'Average Weekly Wage_nabinary',
       'First Hearing Date_nabinary', 'Accident Date

In [10]:
X_train_encoded_bin

,Age at Injury,IME-4 Count,Number of Dependents,Alternative Dispute Resolution_binary,COVID-19 Indicator_binary,Attorney/Representative_binary,Industry Code_encoded_0,Industry Code_encoded_1,WCIO Cause of Injury Code_encoded_0,WCIO Cause of Injury Code_encoded_1,WCIO Nature of Injury Code_encoded_0,WCIO Nature of Injury Code_encoded_1,WCIO Part Of Body Code_encoded_0,WCIO Part Of Body Code_encoded_1,Industry Code_freq,WCIO Cause of Injury Code_freq,WCIO Nature of Injury Code_freq,WCIO Part Of Body Code_freq,Carrier Type Imputed_freq,Carrier Name_freq,Carrier Type_Self-insured Private Entity,Carrier Type_Self-insured Public Entity,Carrier Type_Special Funds,Carrier Type_State Insurance Fund,Carrier Type_nan,C-3 Date_nabinary,Average Weekly Wage_nabinary,First Hearing Date_nabinary,Accident Datemonth,Accident Date_Season_Spring,Accident Date_Season_Summer,Accident Date_Season_Winter,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date Imputed_Accident Date_log,Average Weekly Wage Imputed_log
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5506797,36.0,0.0,4.0,0,0,1,0.585672,0.414328,0.659343,0.340657,0.456437,0.543563,0.601448,0.398552,65354,5507,76889,45299,85050,15062,0.0,1.0,0.0,0.0,0.0,0,1,0,3,1.0,0.0,0.0,4.644391,6.665684,7.472609
5481216,48.0,0.0,5.0,0,1,0,0.455274,0.544726,0.335182,0.664818,0.331479,0.668521,0.326112,0.673888,80703,17307,17799,9693,77075,76840,0.0,0.0,0.0,1.0,0.0,0,1,0,3,1.0,0.0,0.0,4.127134,4.127134,0.693147
5520188,40.0,1.0,6.0,0,0,1,0.455274,0.544726,0.449015,0.550985,0.456437,0.543563,0.592835,0.407165,80703,25939,76889,33274,77075,76840,0.0,0.0,0.0,1.0,0.0,0,0,1,7,0.0,1.0,0.0,2.484907,2.484907,6.933579
5748412,54.0,0.0,1.0,0,0,1,0.629903,0.370097,0.572079,0.427921,0.571953,0.428047,0.371717,0.628283,25977,3073,105965,5254,85050,5721,0.0,1.0,0.0,0.0,0.0,0,0,1,6,0.0,1.0,0.0,1.386294,1.386294,7.120865
5653867,40.0,0.0,0.0,0,0,0,0.359858,0.640142,0.576857,0.423143,0.571953,0.428047,0.544712,0.455288,30487,32209,105965,6855,36539,724,1.0,0.0,0.0,0.0,0.0,1,0,1,2,0.0,0.0,1.0,1.791759,1.791759,6.734818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930802,40.0,0.0,3.0,0,0,0,0.495092,0.504908,0.524503,0.475497,0.337321,0.662679,0.601448,0.398552,12022,22997,418,45299,196754,1393,0.0,0.0,0.0,0.0,0.0,1,1,1,1,0.0,0.0,1.0,3.871201,3.871201,0.693147
5757476,43.0,0.0,4.0,0,0,0,0.439314,0.560686,0.576857,0.423143,0.678726,0.321274,0.472157,0.527843,5652,32209,1852,4687,196754,1998,0.0,0.0,0.0,0.0,0.0,1,1,1,6,0.0,1.0,0.0,2.995732,2.995732,0.693147
5988322,19.0,0.0,0.0,0,0,0,0.455274,0.544726,0.600176,0.399824,0.571953,0.428047,0.384721,0.615279,80703,7971,105965,21899,36539,830,1.0,0.0,0.0,0.0,0.0,1,1,1,5,1.0,0.0,0.0,2.397895,2.397895,0.693147


In [6]:
num_columns_bin = ['Age at Injury', 'IME-4 Count', 
               'Number of Dependents',
               'Industry Code_encoded_0',
               'Industry Code_encoded_1', 
               'WCIO Cause of Injury Code_encoded_0',
               'WCIO Cause of Injury Code_encoded_1',
               'WCIO Nature of Injury Code_encoded_0',
               'WCIO Nature of Injury Code_encoded_1',
               'WCIO Part Of Body Code_encoded_0', 
               'WCIO Part Of Body Code_encoded_1',
               'Industry Code_freq', 
               'WCIO Cause of Injury Code_freq',
               'WCIO Nature of Injury Code_freq',
               'WCIO Part Of Body Code_freq',
               'Carrier Type Imputed_freq', 
               'Carrier Name_freq',
               'Accident Datemonth',
               'Accident Date_Season_Spring', 
               'Accident Date_Season_Summer',
               'Accident Date_Season_Winter',
               'Days_between_Assembly Date_Accident Date_log',
               'Days_between_C-2 Date Imputed_Accident Date_log',
               'Average Weekly Wage Imputed_log']

cat_columns_bin = ['Alternative Dispute Resolution_binary', 
               'COVID-19 Indicator_binary',
               'Attorney/Representative_binary',
               'Carrier Type_Self-insured Private Entity',
               'Carrier Type_Self-insured Public Entity', 
               'Carrier Type_Special Funds',
               'Carrier Type_State Insurance Fund', 
               'C-3 Date_nabinary', 
               'Average Weekly Wage_nabinary',
               'First Hearing Date_nabinary', 
               'Carrier Type_nan'
]

# Create subsets
X_train_num_bin = X_train_encoded_bin[num_columns_bin]
X_train_cat_bin = X_train_encoded_bin[cat_columns_bin]

X_val_num_bin = X_val_encoded_bin[num_columns_bin]
X_val_cat_bin = X_val_encoded_bin[cat_columns_bin]

X_test_num_bin=X_test_encoded_bin[num_columns_bin]
X_test_cat_bin=X_test_encoded_bin[cat_columns_bin]

In [3]:
# num_columns = ['Age at Injury', 'IME-4 Count', 'Number of Dependents',
#                'Days_between_Assembly Date_Accident Date_log',
#                'Days_between_C-2 Date_Accident Date_log', 'Average Weekly Wage_log',
#                'Industry Code_encoded_5. PPD SCH LOSS',
#                'Industry Code_encoded_2. NON-COMP',
#                'Industry Code_encoded_3. MED ONLY',
#                'Industry Code_encoded_4. TEMPORARY',
#                'Industry Code_encoded_1. CANCELLED', 'Industry Code_encoded_8. DEATH',
#                'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
#                'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
#                'WCIO Cause of Injury Code_encoded_2. NON-COMP',
#                'WCIO Cause of Injury Code_encoded_3. MED ONLY',
#                'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
#                'WCIO Cause of Injury Code_encoded_1. CANCELLED',
#                'WCIO Cause of Injury Code_encoded_8. DEATH',
#                'WCIO Cause of Injury Code_encoded_6. PPD NSL',
#                'WCIO Cause of Injury Code_encoded_7. PTD',
#                'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
#                'WCIO Nature of Injury Code_encoded_2. NON-COMP',
#                'WCIO Nature of Injury Code_encoded_3. MED ONLY',
#                'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
#                'WCIO Nature of Injury Code_encoded_1. CANCELLED',
#                'WCIO Nature of Injury Code_encoded_8. DEATH',
#                'WCIO Nature of Injury Code_encoded_6. PPD NSL',
#                'WCIO Nature of Injury Code_encoded_7. PTD',
#                'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
#                'WCIO Part Of Body Code_encoded_2. NON-COMP',
#                'WCIO Part Of Body Code_encoded_3. MED ONLY',
#                'WCIO Part Of Body Code_encoded_4. TEMPORARY',
#                'WCIO Part Of Body Code_encoded_1. CANCELLED',
#                'WCIO Part Of Body Code_encoded_8. DEATH',
#                'WCIO Part Of Body Code_encoded_6. PPD NSL',
#                'WCIO Part Of Body Code_encoded_7. PTD']

# cat_columns = ['Alternative Dispute Resolution_binary', 
#                'COVID-19 Indicator_binary',
#                'Attorney/Representative_binary',
#                'Carrier Type_Self-insured Private Entity',
#                'Carrier Type_Self-insured Public Entity', 
#                'Carrier Type_Special Funds',
#                'Carrier Type_State Insurance Fund', 
#                'C-3 Date_nabinary',
#                'First Hearing Date_nabinary', 
#                'Accident Date_Season_Spring',
#                'Accident Date_Season_Summer', 
#                'Accident Date_Season_Winter']


# # Create subsets
# X_train_num = X_train_encoded[num_columns]
# X_train_cat = X_train_encoded[cat_columns]

# X_val_num = X_val_encoded[num_columns]
# X_val_cat = X_val_encoded[cat_columns]

# X_test_num=X_test_encoded[num_columns]
# X_test_cat=X_test_encoded[cat_columns]

## 3. Scaling

### 3.1 Scale non-binary

In [14]:
# scaler = MinMaxScaler().fit(X_train_num)
# X_train_num_scaled = scaler.transform(X_train_num)
# # print("Parameters fitted:")
# # for feature, min_val, max_val in zip(X_train_num.columns, scaler.data_min_, scaler.data_max_):
#     # print(f"Variable: {feature} | Min: {min_val} | Max: {max_val}")

# # Convert the array to a pandas dataframe
# X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train_encoded.index)
# # X_train_num_scaled.describe().round(2)

In [12]:
# X_val_num_scaled = scaler.transform(X_val_num)
# X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val_encoded.index)
# # X_val_num_scaled.describe().round(2)

In [13]:
# X_test_num_scaled = scaler.transform(X_test_num)
# X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns).set_index(X_test_encoded.index)
# # X_test_num_scaled.describe().round(2)

### 3.2 Scale binary

In [7]:
scaler = MinMaxScaler().fit(X_train_num_bin)
X_train_num_scaled_bin = scaler.transform(X_train_num_bin)
# print("Parameters fitted:")
# for feature, min_val, max_val in zip(X_train_num.columns, scaler.data_min_, scaler.data_max_):
    # print(f"Variable: {feature} | Min: {min_val} | Max: {max_val}")

# Convert the array to a pandas dataframe
X_train_num_scaled_bin = pd.DataFrame(X_train_num_scaled_bin, columns = X_train_num_bin.columns).set_index(X_train_encoded_bin.index)
# X_train_num_scaled.describe().round(2)

In [8]:
X_val_num_scaled_bin = scaler.transform(X_val_num_bin)
X_val_num_scaled_bin = pd.DataFrame(X_val_num_scaled_bin, columns = X_val_num_bin.columns).set_index(X_val_encoded_bin.index)
# X_val_num_scaled.describe().round(2)

In [9]:
X_test_num_scaled_bin = scaler.transform(X_test_num_bin)
X_test_num_scaled_bin = pd.DataFrame(X_test_num_scaled_bin, columns = X_test_num_bin.columns).set_index(X_test_encoded_bin.index)
# X_test_num_scaled.describe().round(2)

## 5. Binary: Feature selection

><strong style="color:#6fa8dc">Add this to a separate python file</strong>


### 5.1 Filter Categorical Features

#### 5.1.1 Chi-square

In [13]:
import pandas as pd  # For creating and handling DataFrames
import numpy as np  # For numerical operations
from sklearn.feature_selection import chi2  # For Chi-square test
from scipy.stats import chi2_contingency  # For Cramér's V calculation

In [11]:
def cramers_v(X, y):
    """
    Calculate Cramér's V for a categorical feature and a target.
    :param X: Categorical feature (Pandas Series or array-like)
    :param y: Target variable (Pandas Series or array-like)
    :return: Cramér's V value
    """
    # Create the contingency table
    df_contingency = pd.crosstab(X, y)

    # Perform chi-square test
    chi2, p, dof, expected = chi2_contingency(df_contingency.values)

    # Calculate Cramér's V
    n = df_contingency.sum().sum()  # Total number of observations
    min_dim = min(df_contingency.shape) - 1  # Min between number of rows and columns - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula

    return cramers_v


In [14]:
# Perform Chi-square test
chi2_values, p_values = chi2(X_train_cat_bin, y_train_bin)

# Create DataFrame for Chi-square results
chi2_results = pd.DataFrame({
    'Column': X_train_cat_bin.columns,
    'Chi2': chi2_values.round(5),
    'p-value': p_values.round(5)
})

# Calculate Cramér's V for binary target
cramers_v_values = []
for var in X_train_cat_bin.columns:
    v = cramers_v(X_train_cat_bin[var], y_train_bin)
    cramers_v_values.append(v)

# Add Cramér's V to DataFrame
chi2_results['Cramér\'s V'] = cramers_v_values

# Filter important features
chi2_important_features = chi2_results[(chi2_results['p-value'] < 0.05) & (chi2_results['Cramér\'s V'] >= 0.15)]

list_features_chi2_cramer = chi2_important_features['Column'].values

print(chi2_important_features)

                           Column         Chi2  p-value  Cramér's V
2  Attorney/Representative_binary  78009.76607      0.0    0.535952
7               C-3 Date_nabinary  23953.69374      0.0    0.429801
8    Average Weekly Wage_nabinary  86212.24203      0.0    0.770968
9     First Hearing Date_nabinary  25813.45742      0.0    0.498054


#### 5.1.2 Mutual Information

In [15]:
from sklearn.feature_selection import mutual_info_classif

# Calculate mutual information for categorical features
mi_scores = mutual_info_classif(X_train_cat_bin, y_train_bin, discrete_features=True)

# Create DataFrame of results
mi_results = pd.DataFrame({
    'Feature': X_train_cat_bin.columns,
    'Mutual Information': mi_scores
}).sort_values(by='Mutual Information', ascending=False)

mi_important_features = mi_results[mi_results['Mutual Information'] > 0.05]

list_features_mi = mi_results[mi_results['Mutual Information'] > 0.05]['Feature'].values

print(mi_important_features)

                          Feature  Mutual Information
8    Average Weekly Wage_nabinary            0.370264
2  Attorney/Representative_binary            0.157423
9     First Hearing Date_nabinary            0.138322
7               C-3 Date_nabinary            0.096962


In [16]:
# Select features that appear in at least one of the important feature sets
selected_features = list(set(list_features_chi2_cramer) | set(list_features_mi)) 
selected_features

['C-3 Date_nabinary',
 'First Hearing Date_nabinary',
 'Average Weekly Wage_nabinary',
 'Attorney/Representative_binary']

In [17]:
# Keep only selected important features in the datasets
X_train_cat_filtered_bin = X_train_cat_bin[selected_features]
X_val_cat_filtered_bin = X_val_cat_bin[selected_features]
X_test_cat_filtered_bin = X_test_cat_bin[selected_features]

### 5.2 Filter Numerical Features

In [18]:
X_train_num_scaled_bin.var().sort_values(ascending=False)

Accident Date_Season_Summer                        0.190598
Accident Date_Season_Winter                        0.190024
Accident Date_Season_Spring                        0.181228
Average Weekly Wage Imputed_log                    0.158004
Carrier Name_freq                                  0.142667
WCIO Nature of Injury Code_freq                    0.136735
Industry Code_freq                                 0.123001
Number of Dependents                               0.111300
Carrier Type Imputed_freq                          0.103530
Accident Datemonth                                 0.099293
WCIO Part Of Body Code_freq                        0.098325
Industry Code_encoded_1                            0.096085
Industry Code_encoded_0                            0.096085
WCIO Cause of Injury Code_freq                     0.080853
WCIO Part Of Body Code_encoded_1                   0.057635
WCIO Part Of Body Code_encoded_0                   0.057635
Age at Injury                           

> None with variance zero

In [19]:
import pandas as pd

# Step 1: Calculate the Spearman correlation matrix for numerical features
cor_spearman = X_train_num_scaled_bin.corr(method='spearman')

# Step 2: Flatten the correlation matrix and reset the index
correlation_pairs = cor_spearman.unstack().reset_index()

# Step 3: Rename columns for clarity
correlation_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# Step 4: Filter the table for correlations > 0.8 or < -0.8 and exclude self-correlations (diagonal)
strong_correlations = correlation_pairs[
    ((correlation_pairs['Correlation'] > 0.8) | (correlation_pairs['Correlation'] < -0.8)) & 
    (correlation_pairs['Feature_1'] != correlation_pairs['Feature_2'])
]

# Step 5: Drop duplicate pairs (keep unique pairs only)
strong_correlations = strong_correlations.sort_values(by='Correlation', ascending=False).drop_duplicates(subset=['Feature_1', 'Feature_2'])

# Step 6: Display the table
strong_correlations


,Feature_1,Feature_2,Correlation
526,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date Imputed_Accident Date_log,0.954048
549,Days_between_C-2 Date Imputed_Accident Date_log,Days_between_Assembly Date_Accident Date_log,0.954048
76,Industry Code_encoded_0,Industry Code_encoded_1,-1.000000
99,Industry Code_encoded_1,Industry Code_encoded_0,-1.000000
126,WCIO Cause of Injury Code_encoded_0,WCIO Cause of Injury Code_encoded_1,-1.000000
149,WCIO Cause of Injury Code_encoded_1,WCIO Cause of Injury Code_encoded_0,-1.000000
176,WCIO Nature of Injury Code_encoded_0,WCIO Nature of Injury Code_encoded_1,-1.000000
199,WCIO Nature of Injury Code_encoded_1,WCIO Nature of Injury Code_encoded_0,-1.000000
226,WCIO Part Of Body Code_encoded_0,WCIO Part Of Body Code_encoded_1,-1.000000
249,WCIO Part Of Body Code_encoded_1,WCIO Part Of Body Code_encoded_0,-1.000000


In [20]:
# List of features to drop from X_train_num_scaled
num_features_to_drop = [
    'Days_between_C-2 Date Imputed_Accident Date_log',
    'Industry Code_encoded_0',
    'WCIO Cause of Injury Code_encoded_0',
    'WCIO Nature of Injury Code_encoded_0',
    'WCIO Part Of Body Code_encoded_0'
]

# Drop features
X_train_num_scaled_filtered_bin = X_train_num_scaled_bin.drop(columns=num_features_to_drop)
X_val_num_scaled_filtered_bin = X_val_num_scaled_bin.drop(columns=num_features_to_drop)
X_test_num_scaled_filtered_bin = X_test_num_scaled_bin.drop(columns=num_features_to_drop)

# Verify the remaining columns
print("Remaining features in X_train_num_scaled:", X_train_num_scaled_filtered_bin.columns.tolist())


Remaining features in X_train_num_scaled: ['Age at Injury', 'IME-4 Count', 'Number of Dependents', 'Industry Code_encoded_1', 'WCIO Cause of Injury Code_encoded_1', 'WCIO Nature of Injury Code_encoded_1', 'WCIO Part Of Body Code_encoded_1', 'Industry Code_freq', 'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_freq', 'WCIO Part Of Body Code_freq', 'Carrier Type Imputed_freq', 'Carrier Name_freq', 'Accident Datemonth', 'Accident Date_Season_Spring', 'Accident Date_Season_Summer', 'Accident Date_Season_Winter', 'Days_between_Assembly Date_Accident Date_log', 'Average Weekly Wage Imputed_log']


### 5.3 Combine the filtered datasets

In [22]:
# Combine the filtered datasets
X_train_bin = pd.concat([X_train_cat_filtered_bin, X_train_num_scaled_filtered_bin], axis=1)
X_val_bin =  pd.concat([X_val_cat_filtered_bin, X_val_num_scaled_filtered_bin], axis=1)
X_test_bin =  pd.concat([X_test_cat_filtered_bin, X_test_num_scaled_filtered_bin], axis=1)


# Verify the shape of the combined dataset
print("Shape of combined X_train:", X_train_bin.shape)


Shape of combined X_train: (396097, 23)


> Use this for the binary, as we will be dropping later for multiclass in minority

### 5.4 Feature selection

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import numpy as np

# Initialize the Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)

# Sample 1% of the data
sample_size = int(0.01 * len(X_train_bin))  # 1% of the dataset
X_train_sample = X_train_bin.sample(n=sample_size, random_state=42)
y_train_sample = y_train_bin.sample(n=sample_size, random_state=42)

# Loop over different numbers of features
feature_counts = range(1, X_train_sample.shape[1] + 1)
scores = []

# Loop through each feature count and evaluate the model's performance
for n in feature_counts:
    rfe = RFE(log_reg_model, n_features_to_select=n)
    rfe.fit(X_train_sample, y_train_sample)
    
    # Get selected features
    selected_features = X_train_sample.columns[rfe.support_]
    
    # Evaluate performance using cross-validation with F1 score as the metric
    score = cross_val_score(log_reg_model, X_train_sample[selected_features], y_train_sample, cv=5, scoring='f1_macro').mean()
    scores.append(score)

# Find the number of features that gives the highest F1 score
best_n_features = feature_counts[np.argmax(scores)]
print(f"Optimal number of features: {best_n_features}")


Optimal number of features: 19


In [70]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Assuming X_train_cat and y_train are your features and target

# Step 1: Take a smaller sample (e.g., 10% of the data)
sample_size = int(0.01 * len(X_train_binary))  # 10% of the dataset
X_train_sample = X_train_binary.sample(n=sample_size, random_state=42)
y_train_sample = y_binary.sample(n=sample_size, random_state=42)

# Step 2: Initialize Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 3: Apply RFE to the sample data
rfe = RFE(rf_model, n_features_to_select=5)  # Select top 5 features (adjust if needed)
rfe.fit(X_train_sample, y_train_sample)

# Step 4: Get the selected features
selected_features_rfe = X_train_sample.columns[rfe.support_]

# Print selected features
print("Selected Features after RFE with Random Forest:")
print(selected_features_rfe)

# Optionally, you can use the selected features to transform the dataset
X_train_selected_rfe = X_train_sample[selected_features_rfe]


In [23]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=47)

# Fit the model on X_train and y_train_encoded
rf_model.fit(X_train_bin, y_train_bin)

# Get feature importances
importances = rf_model.feature_importances_

# Sort feature importances in descending order
important_indices = importances.argsort()[::-1]

# Print the feature importances
print("Feature importances:", importances)
print("Sorted feature indices:", important_indices)


Feature importances: [0.0244034  0.04960444 0.29075435 0.06925285 0.04081886 0.04807678
 0.02330087 0.01904752 0.02579563 0.02201104 0.02611799 0.01658626
 0.02140705 0.01249304 0.02084179 0.00822729 0.03412091 0.02259905
 0.00424403 0.00453387 0.00421157 0.04037683 0.17117458]
Sorted feature indices: [ 2 22  3  1  5  4 21 16 10  8  0  6 17  9 12 14  7 11 13 15 19 18 20]


In [24]:
import numpy as np

# Calculate cumulative sum of feature importances
cumulative_importance = np.cumsum(importances[important_indices])

# Find the index where cumulative importance exceeds 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The number of features that explain 90% of the importance
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# Select the top features based on this number
random_forest_features = X_train_bin.columns[important_indices[:num_features_90]]
print(f"Selected features that explain 90% of cumulative importance: {random_forest_features}")


Number of features explaining 90% of cumulative importance: 15
Selected features that explain 90% of cumulative importance: Index(['Average Weekly Wage_nabinary', 'Average Weekly Wage Imputed_log',
       'Attorney/Representative_binary', 'First Hearing Date_nabinary',
       'IME-4 Count', 'Age at Injury',
       'Days_between_Assembly Date_Accident Date_log', 'Carrier Name_freq',
       'WCIO Part Of Body Code_encoded_1',
       'WCIO Cause of Injury Code_encoded_1', 'C-3 Date_nabinary',
       'Number of Dependents', 'Accident Datemonth',
       'WCIO Nature of Injury Code_encoded_1',
       'WCIO Cause of Injury Code_freq'],
      dtype='object')


In [25]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5)  # Cross-validation to select optimal alpha
lasso.fit(X_train_bin, y_train_bin)
lasso_features = X_train_bin.columns[lasso.coef_ != 0]
print("Selected features by Lasso:", lasso_features)

Selected features by Lasso: Index(['C-3 Date_nabinary', 'First Hearing Date_nabinary',
       'Average Weekly Wage_nabinary', 'Attorney/Representative_binary',
       'Age at Injury', 'IME-4 Count', 'Industry Code_encoded_1',
       'WCIO Cause of Injury Code_encoded_1',
       'WCIO Nature of Injury Code_encoded_1',
       'WCIO Part Of Body Code_encoded_1', 'Industry Code_freq',
       'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_freq',
       'WCIO Part Of Body Code_freq', 'Carrier Type Imputed_freq',
       'Carrier Name_freq', 'Accident Datemonth',
       'Accident Date_Season_Summer', 'Accident Date_Season_Winter',
       'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage Imputed_log'],
      dtype='object')


In [26]:
from sklearn.linear_model import RidgeCV
import numpy as np

# Initialize Ridge regression model with cross-validation
ridge_model = RidgeCV(cv=5)

# Fit the model on the training data
ridge_model.fit(X_train_bin, y_train_bin)

# Get the feature coefficients
coefficients = ridge_model.coef_

# Set a threshold to select features (e.g., absolute coefficient > 0.01)
threshold = 0.001
ridge_features = X_train_bin.columns[np.abs(coefficients) > threshold]

print(f"Selected features using Ridge regression: {ridge_features}")


Selected features using Ridge regression: Index(['C-3 Date_nabinary', 'First Hearing Date_nabinary',
       'Average Weekly Wage_nabinary', 'Attorney/Representative_binary',
       'Age at Injury', 'IME-4 Count', 'Number of Dependents',
       'Industry Code_encoded_1', 'WCIO Cause of Injury Code_encoded_1',
       'WCIO Nature of Injury Code_encoded_1',
       'WCIO Part Of Body Code_encoded_1', 'Industry Code_freq',
       'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_freq',
       'WCIO Part Of Body Code_freq', 'Carrier Type Imputed_freq',
       'Carrier Name_freq', 'Accident Datemonth',
       'Accident Date_Season_Spring', 'Accident Date_Season_Summer',
       'Accident Date_Season_Winter',
       'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage Imputed_log'],
      dtype='object')


In [33]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

# Initialize the XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, random_state=47, use_label_encoder=False, eval_metric="mlogloss")

# Fit the model on X_train and y_train_encoded
xgb_model.fit(X_train_bin, y_train_bin)

# Get feature importances
importances = xgb_model.feature_importances_

# Sort feature importances in descending order
important_indices = np.argsort(importances)[::-1]

# Get feature names from the column names of X_train_bin
feature_names = X_train_bin.columns

# Create a DataFrame to view the feature names and their importance scores
feature_importance_df = pd.DataFrame({
    'Feature': feature_names[important_indices],
    'Importance': importances[important_indices]
})

# Print the feature importances
print(feature_importance_df)


c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:31:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


                                         Feature  Importance
0                   Average Weekly Wage_nabinary    0.948941
1                 Attorney/Representative_binary    0.028553
2                Average Weekly Wage Imputed_log    0.003426
3                                    IME-4 Count    0.003214
4                    First Hearing Date_nabinary    0.002966
5           WCIO Nature of Injury Code_encoded_1    0.001893
6                      Carrier Type Imputed_freq    0.001778
7                              C-3 Date_nabinary    0.001401
8                              Carrier Name_freq    0.000976
9            WCIO Cause of Injury Code_encoded_1    0.000813
10              WCIO Part Of Body Code_encoded_1    0.000779
11                       Industry Code_encoded_1    0.000776
12               WCIO Nature of Injury Code_freq    0.000738
13                            Industry Code_freq    0.000629
14  Days_between_Assembly Date_Accident Date_log    0.000484
15                   WCI

In [31]:
import numpy as np

# Calculate cumulative sum of feature importances
cumulative_importance = np.cumsum(importances[important_indices])

# Find the index where cumulative importance exceeds 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The number of features that explain 90% of the importance
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# Select the top features based on this number
xgb_feautres = X_train_bin.columns[important_indices[:num_features_90]]
print(f"Selected features that explain 90% of cumulative importance: {xgb_feautres}")


Number of features explaining 90% of cumulative importance: 1
Selected features that explain 90% of cumulative importance: Index(['Average Weekly Wage_nabinary'], dtype='object')


In [37]:
# Convert all feature sets to sets
rf_set = set(random_forest_features)
lasso_set = set(lasso_features)
ridge_set = set(ridge_features)

# Find features selected by at least three methods
final_features_set = (
    ( lasso_set & rf_set) |
    ( lasso_set & ridge_set) | (ridge_set & rf_set)
 
)


# Convert to sorted list
final_features = sorted(final_features_set)

# Display the final feature set
print(f"Features selected by 3 or more methods ({len(final_features)} features):")
print(final_features)


Features selected by 3 or more methods (21 features):
['Accident Date_Season_Summer', 'Accident Date_Season_Winter', 'Accident Datemonth', 'Age at Injury', 'Attorney/Representative_binary', 'Average Weekly Wage Imputed_log', 'Average Weekly Wage_nabinary', 'C-3 Date_nabinary', 'Carrier Name_freq', 'Carrier Type Imputed_freq', 'Days_between_Assembly Date_Accident Date_log', 'First Hearing Date_nabinary', 'IME-4 Count', 'Industry Code_encoded_1', 'Industry Code_freq', 'WCIO Cause of Injury Code_encoded_1', 'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_encoded_1', 'WCIO Nature of Injury Code_freq', 'WCIO Part Of Body Code_encoded_1', 'WCIO Part Of Body Code_freq']


In [48]:
final_features=[ 
                'Accident Date_Season_Winter', 
                'Accident Datemonth', 
                'Age at Injury', 
                'Attorney/Representative_binary', 
                'Average Weekly Wage Imputed_log',
                'Average Weekly Wage_nabinary', 
                'C-3 Date_nabinary', 
                'Carrier Name_freq', 
                'Carrier Type Imputed_freq', 
                'Days_between_Assembly Date_Accident Date_log', 
                'First Hearing Date_nabinary', 
                'IME-4 Count', 
                'Industry Code_encoded_1', 
                'Industry Code_freq', 
                'WCIO Cause of Injury Code_encoded_1', 
                'WCIO Cause of Injury Code_freq', 
                'WCIO Nature of Injury Code_encoded_1',
                'WCIO Nature of Injury Code_freq',
                'WCIO Part Of Body Code_encoded_1']


In [49]:
X_train_bin_filter = X_train_bin[final_features]
X_val_bin_filter = X_val_bin[final_features]
X_test_bin_filter = X_test_bin[final_features]

In [55]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to balance the dataset
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_bin_filter, y_train_bin)

# Check the distribution after resampling
print(f"Original class distribution: {Counter(y_train_bin)}")
print(f"Resampled class distribution: {Counter(y_train_resampled)}")


Original class distribution: Counter({1: 202245, 0: 193852})
Resampled class distribution: Counter({0: 202245, 1: 202245})


In [65]:
def custom_loss(y_pred, dtrain):
    """
    Custom loss function penalizing false negatives for class 0.
    Parameters:
    - y_pred: Predicted values (log-odds).
    - dtrain: DMatrix containing true labels.

    Returns:
    - grad: Gradient of the loss.
    - hess: Hessian of the loss.
    """
    y_true = dtrain.get_label()  # Extract true labels from DMatrix
    y_pred = 1 / (1 + np.exp(-y_pred))  # Convert log-odds to probabilities

    # Adjust weights for class 0
    weight = np.where(y_true == 0, 2.0, 1.0)  # Double penalty for class 0 FN

    grad = (y_pred - y_true) * weight  # Gradient
    hess = y_pred * (1 - y_pred) * weight  # Hessian
    return grad, hess


In [66]:
import xgboost as xgb

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train_bin_filter, label=y_train_bin)
dval = xgb.DMatrix(X_val_bin_filter, label=y_val_bin)

# Parameters for XGBoost
params = {
    "objective": "binary:logistic",  # Use binary classification
    "eval_metric": "logloss",  # Standard evaluation metric
    "eta": 0.1,  # Learning rate
    "max_depth": 6,  # Maximum depth of trees
}

# Train the model with custom loss function
model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    obj=custom_loss,  # Custom loss function
    evals=[(dval, "validation")],
    early_stopping_rounds=10,  # Stop if no improvement
)


[0]	validation-logloss:0.63529
[1]	validation-logloss:0.58758
[2]	validation-logloss:0.54760
[3]	validation-logloss:0.51395
[4]	validation-logloss:0.48509
[5]	validation-logloss:0.46048
[6]	validation-logloss:0.43906
[7]	validation-logloss:0.42051
[8]	validation-logloss:0.40426
[9]	validation-logloss:0.39008
[10]	validation-logloss:0.37761
[11]	validation-logloss:0.36662
[12]	validation-logloss:0.35700
[13]	validation-logloss:0.34840
[14]	validation-logloss:0.34080
[15]	validation-logloss:0.33405
[16]	validation-logloss:0.32783
[17]	validation-logloss:0.32235
[18]	validation-logloss:0.31755
[19]	validation-logloss:0.31329
[20]	validation-logloss:0.30931
[21]	validation-logloss:0.30586
[22]	validation-logloss:0.30284
[23]	validation-logloss:0.30007
[24]	validation-logloss:0.29754
[25]	validation-logloss:0.29538
[26]	validation-logloss:0.29321
[27]	validation-logloss:0.29132
[28]	validation-logloss:0.28961
[29]	validation-logloss:0.28821
[30]	validation-logloss:0.28691
[31]	validation-lo

In [67]:
# Predict probabilities
y_proba = model.predict(xgb.DMatrix(X_val_bin_filter))

# Convert probabilities to binary predictions using a threshold (default is 0.5)
threshold = 0.5
y_pred = (y_proba > threshold).astype(int)

# Evaluate
from sklearn.metrics import classification_report, accuracy_score, f1_score

print("Accuracy:", accuracy_score(y_val_bin, y_pred))
print("F1 Score:", f1_score(y_val_bin, y_pred, average="macro"))
print(classification_report(y_val_bin, y_pred))


Accuracy: 0.8946457607433217
F1 Score: 0.8944667576892235
              precision    recall  f1-score   support

           0       0.92      0.87      0.89     84881
           1       0.88      0.92      0.90     87319

    accuracy                           0.89    172200
   macro avg       0.90      0.89      0.89    172200
weighted avg       0.90      0.89      0.89    172200



In [61]:
import xgboost as xgb

model = xgb.XGBClassifier(eval_metric="mlogloss")
model.fit(X_train_resampled, y_train_resampled)

# 3. Make predictions
y_pred = model.predict(X_val_bin_filter)  # Assuming X_test is available


# Evaluate the model
accuracy = accuracy_score(y_val_bin, y_pred)
f1_macro = f1_score(y_val_bin, y_pred, average='macro')  # Macro-averaged F1 score

# Print individual model results
print(f"XGBClassifier Accuracy: {accuracy:.4f}")
print(f"XGBClassifier Macro F1 Score: {f1_macro:.4f}")
print(classification_report(y_val_bin, y_pred))  # Detailed report including precision, recall, and F1 score per class
print("-" * 50)

# Display the summary of results
print("\nModel Evaluation Results Summary:")
print(f"XGBClassifier: Accuracy = {accuracy:.4f}, Macro F1 Score = {f1_macro:.4f}")


XGBClassifier Accuracy: 0.9018
XGBClassifier Macro F1 Score: 0.9012
              precision    recall  f1-score   support

           0       0.96      0.84      0.89     84881
           1       0.86      0.97      0.91     87319

    accuracy                           0.90    172200
   macro avg       0.91      0.90      0.90    172200
weighted avg       0.91      0.90      0.90    172200

--------------------------------------------------

Model Evaluation Results Summary:
XGBClassifier: Accuracy = 0.9018, Macro F1 Score = 0.9012


In [58]:
from sklearn.ensemble import VotingClassifier

ensemble_model = VotingClassifier(
    estimators=[
        ('xgb', XGBClassifier(eval_metric="mlogloss")),
        ('rf', RandomForestClassifier())
    ],
    voting='soft'  # Use probabilities for majority decision
)
ensemble_model.fit(X_train_bin_filter, y_train_bin)


KeyboardInterrupt: 

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize Logistic Regression
log_reg_model = LogisticRegression(random_state=42)

# Fit the model on training data
log_reg_model.fit(X_train_bin_filter, y_train_bin)

# Make predictions
y_pred_log_reg = log_reg_model.predict(X_val_bin_filter)

# Evaluate the model
accuracy_log_reg = accuracy_score(y_val_bin, y_pred_log_reg)
f1_macro_log_reg = f1_score(y_val_bin, y_pred_log_reg, average='macro')

# Print results
print(f"Logistic Regression Accuracy: {accuracy_log_reg:.4f}")
print(f"Logistic Regression Macro F1 Score: {f1_macro_log_reg:.4f}")
print(classification_report(y_val_bin, y_pred_log_reg))


Logistic Regression Accuracy: 0.8875
Logistic Regression Macro F1 Score: 0.8865
              precision    recall  f1-score   support

           0       0.96      0.81      0.88     84881
           1       0.84      0.97      0.90     87319

    accuracy                           0.89    172200
   macro avg       0.90      0.89      0.89    172200
weighted avg       0.90      0.89      0.89    172200



## 6. Train model binary

In [1]:
from sklearn.metrics import accuracy_score, classification_report, f1_score
from xgboost import XGBClassifier

# Train-test sp
# Define the XGBClassifier
model = XGBClassifier()

# Train the model
print(f"Training XGBClassifier...")
model.fit(X_train_bin_filter, y_train_bin)

# Predict on the validation data
y_pred = model.predict(X_val_bin_filter)

# Evaluate the model
accuracy = accuracy_score(y_val_encoded, y_pred)
f1_macro = f1_score(y_val_encoded, y_pred, average='macro')  # Macro-averaged F1 score

# Print individual model results
print(f"XGBClassifier Accuracy: {accuracy:.4f}")
print(f"XGBClassifier Macro F1 Score: {f1_macro:.4f}")
print(classification_report(y_val_encoded, y_pred))  # Detailed report including precision, recall, and F1 score per class
print("-" * 50)

# Display the summary of results
print("\nModel Evaluation Results Summary:")
print(f"XGBClassifier: Accuracy = {accuracy:.4f}, Macro F1 Score = {f1_macro:.4f}")


Training XGBClassifier...


NameError: name 'X_train_bin_filter' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Split the data into training and test sets


# Initialize models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    # "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(max_iter=1000),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(silent=True)
}

# Dictionary to store F1 scores
f1_scores = {}

# Loop over models, train, and evaluate F1 score (macro)
for model_name, model in models.items():
    model.fit(X_train_bin_filter, y_train_bin)
    y_pred = model.predict(X_val_bin_filter)
    
    # Calculate F1 score (macro)
    f1 = f1_score(y_val_bin, y_pred, average='macro')
    f1_scores[model_name] = f1

# Print the F1 scores
for model_name, f1 in f1_scores.items():
    print(f"{model_name}: F1 Score (Macro): {f1:.4f}")


c:\Users\emano\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
